In [1]:
import os
import ROOT

Welcome to JupyROOT 6.30/02


In [86]:
ERA = "2018"
REGION = "EE"
#cuts = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
#cuts = [0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89]
#cuts = [0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84]
cuts = [0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59,
        0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69]

In [87]:
def get_fakerate(cut):
    # get histograms
    path = f"results/{ERA}/entries.{REGION}.{str(cut).replace('.', 'p')}.root"
    f = ROOT.TFile.Open(path)
    h_ljet_loose = f.Get("ljet_loose"); h_ljet_loose.SetDirectory(0)
    h_cjet_loose = f.Get("cjet_loose"); h_cjet_loose.SetDirectory(0)
    h_bjet_loose = f.Get("bjet_loose"); h_bjet_loose.SetDirectory(0)

    h_ljet_tight = f.Get("ljet_tight"); h_ljet_tight.SetDirectory(0)
    h_cjet_tight = f.Get("cjet_tight"); h_cjet_tight.SetDirectory(0)
    h_bjet_tight = f.Get("bjet_tight"); h_bjet_tight.SetDirectory(0)
    f.Close()
    
    h_all_loose = h_ljet_loose.Clone("all_loose")
    h_all_loose.Add(h_cjet_loose)
    h_all_loose.Add(h_bjet_loose)
    
    h_all_tight = h_ljet_tight.Clone("all_tight")
    h_all_tight.Add(h_cjet_tight)
    h_all_tight.Add(h_bjet_tight)
    
    # calculate fake rate
    fake_ljet = h_ljet_tight.Clone("fake_ljet"); fake_ljet.Divide(h_ljet_loose)
    fake_cjet = h_cjet_tight.Clone("fake_cjet"); fake_cjet.Divide(h_cjet_loose)
    fake_bjet = h_bjet_tight.Clone("fake_bjet"); fake_bjet.Divide(h_bjet_loose)
    fake_all = h_all_tight.Clone("fake_all"); fake_all.Divide(h_all_loose)

    fake_all.SetStats(0)
    fake_all.SetLineColor(ROOT.kBlack)
    fake_all.SetLineWidth(4)
    fake_all.GetXaxis().SetRangeUser(10., 200.)
    fake_all.GetYaxis().SetRangeUser(0., 1.)
    fake_all.GetXaxis().SetLabelSize(0)
    fake_all.GetYaxis().SetTitle("fake rate")
    
    fake_ljet.SetStats(0)
    fake_ljet.SetLineColor(ROOT.kRed)
    fake_ljet.SetLineWidth(4)

    fake_cjet.SetStats(0)
    fake_cjet.SetLineColor(ROOT.kGreen)
    fake_cjet.SetLineWidth(4)

    fake_bjet.SetStats(0)
    fake_bjet.SetLineColor(ROOT.kBlue)
    fake_bjet.SetLineWidth(4)
    
    return fake_all, fake_ljet, fake_cjet, fake_bjet

In [88]:
# get histograms and make ratios
def get_ratio(fake_all, fake_ljet, fake_cjet, fake_bjet):
    ratio_ljet = fake_ljet.Clone("ratio_ljet"); ratio_ljet.Divide(fake_all)
    ratio_bjet = fake_bjet.Clone("ratio_bjet"); ratio_bjet.Divide(fake_all)
    ratio_cjet = fake_cjet.Clone("ratio_cjet"); ratio_cjet.Divide(fake_all)

    ratio_ljet.GetXaxis().SetTitle("p_{T}^{corr} [GeV]")
    ratio_ljet.GetXaxis().SetTitleSize(0.1)
    ratio_ljet.GetXaxis().SetTitleOffset(0.9)
    ratio_ljet.GetXaxis().SetLabelSize(0.08)
    ratio_ljet.GetYaxis().SetTitle("fr / fr_{all}")
    ratio_ljet.GetYaxis().SetTitleSize(0.1)
    ratio_ljet.GetYaxis().SetTitleOffset(0.45)
    ratio_ljet.GetYaxis().SetLabelSize(0.08)
    ratio_ljet.GetYaxis().CenterTitle()
    ratio_ljet.GetYaxis().SetRangeUser(0.5, 1.5)
    
    return ratio_ljet, ratio_cjet, ratio_bjet

In [89]:
for cut in cuts:
    fake_all, fake_ljet, fake_cjet, fake_bjet = get_fakerate(cut)
    ratio_ljet, ratio_cjet, ratio_bjet = get_ratio(fake_all, fake_ljet, fake_cjet, fake_bjet)
    
    # prepare canvas and legend
    c = ROOT.TCanvas("c", "", 1600, 1800)
    pad_up = ROOT.TPad("pad_up", "", 0, 0.25, 1, 1)
    pad_up.SetBottomMargin(0.02)
    pad_down = ROOT.TPad("pad_down", "", 0, 0, 1, 0.25)
    pad_down.SetTopMargin(0.05)
    pad_down.SetBottomMargin(0.3)
    pad_down.SetGrid()
    legend = ROOT.TLegend(0.65, 0.50, 0.9, 0.87)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)

    info = ROOT.TLatex()
    info.SetTextSize(0.035)
    info.SetTextFont(42)
    logo = ROOT.TLatex()
    logo.SetTextSize(0.04)
    logo.SetTextFont(61)
    extra_logo = ROOT.TLatex()
    extra_logo.SetTextSize(0.035)
    extra_logo.SetTextFont(52)

    legend.AddEntry(fake_all, "ALL", "lep")
    legend.AddEntry(fake_ljet, "L", "lep")
    legend.AddEntry(fake_cjet, "C", "lep")
    legend.AddEntry(fake_bjet, "B", "lep")
    
    pad_up.cd()
    fake_all.Draw()
    fake_ljet.Draw("same")
    fake_cjet.Draw("same")
    fake_bjet.Draw("same")
    fake_all.Draw("same")
    legend.Draw("same")
    info.DrawLatexNDC(0.80, 0.91, "(13TeV)")
    logo.DrawLatexNDC(0.1, 0.91, "CMS")
    extra_logo.DrawLatexNDC(0.18, 0.91, "Simulation, Work in progress")
    pad_up.RedrawAxis()

    pad_down.cd()
    ratio_ljet.Draw()
    ratio_cjet.Draw("same")
    ratio_bjet.Draw("same")
    pad_down.RedrawAxis()

    c.cd()
    pad_up.Draw()
    pad_down.Draw()
    
    outpath = f"results/{ERA}/plots/fakerate.{REGION}.{str(cut).replace('.', 'p')}.png"
    os.makedirs(os.path.dirname(outpath), exist_ok=True)
    c.SaveAs(outpath)
    

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file results/2018/plots/fakerate.EE.0p5.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file results/2018/plots/fakerate.EE.0p51.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file results/2018/plots/fakerate.EE.0p52.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file results/2018/plots/fakerate.EE.0p53.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file results/2018/plots/fakerate.EE.0p54.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file results/2018/plots/fakerate.EE.0p55.png has been created
Warning in <TCanvas::Constructor>: 